**Algorytm KNN**

W najprostszej wersji algorytm KNN jest algorytmem klasyfikującym. Klasyfikacja obiektu polega na znalezieniu $K$ najbliższych sąsiadów względem wybranej przez nas metryki oraz wybraniu najliczniejszej klasy spośród tych sąsiadów.

Algorytm KNN może też być stosowany w wersji regresyjnej. W tej wersji obiektowi przypisujemy wartość, która jest średnią wartości $K$ najbliższych sąsiadów.

**Teoretyczne ograniczenia algorytmu KNN**

$C_1, \ldots, C_M$ - klasy, do których chcemy przypisywać obiekty.

$X$ - rodzina obiektów, które chcemy klasyfikować.

W teoretycznych rozważaniach przyjmuje się, że każdy obiekt ma zadany 'prawdziwy' rozkład prawdopodobieństw ($\hat{P}(C_i | x)$ dla $x \in X$) przynależności do klas.
Zadaniem klasyfikatorów jest tak naprawdę aproksymacja tych rozkładów.

Przez **błąd** klasyfikatora na obiekcie $x$ rozumiemy prawdopodobieństwo przypisania $x$ do niepoprawnej klasy
$$\texttt{Err}_x = \sum_{i = 1}^{M} (C_i \neq \tilde{C}(x))\hat{P}(C_i|x),$$
gdzie $\tilde{C}(x)$ to klasa, którą nasz klasyfikator przypisuje do $x$.

Przez **współczynnik błędu** klasyfikatora rozumiemy oczekiwaną wartość błędu, gdy obiekt $x$ jest losowo wybrany z rodziny $X$. Niekoniecznie musi się to dziać z jednostajnym prawdopodobieństwem, ale na potrzeby naszych rozważań przyjmujemy takie założenie.
$$\texttt{Err} = E[\texttt{Err}_x].$$

Przez **Bayesowski współczynnik błędu** rozumiemy najniższy możliwy współczynnik błędu. Jest on równy współczynnikowi błędu 'idealnego' klasyfikatora, który zna prawdziwe rozkłady $\hat{P}(C_i | x)$ i każdy element po prostu przypisuje do najbardziej prawdopodobnej klasy.

W takiej sytuacji błąd na obiekcie $x$ upraszcza się do
$$\texttt{Err}_x = 1 - \max_i \hat{P}(C_i | x),$$

a więc Bayesowski współczynnik błędu wyrażony jest przez
$$B^* = E_x\bigg[1 - \max_i \hat{P}(C_i | x)\bigg].$$

Po dokonaniu kilku zdroworozsądkowych założeń można udowodnić, że błąd klasyfikatora KNN jest co najwyżej dwa razy większy niż błąd Bayesowski.

Dokładniejsze ograniczenie wynosi
$$B_{kNN} \leq B^*\bigg(2 - \frac{M B^*}{M - 1}\bigg).$$

**Złożoność obliczeniowa**

$T$ - rozmiar zbioru treningowego

$S$ - rozmiar zbioru testowego

$K$ - parametr KNN

W naiwnej implementacji KNN porównujemy odległości obiektów $T \cdot S \cdot K$ razy. To dużo.

Można tą ilość łatwo zmniejszyć wykorzystując *kopiec* (struktura danych) do $T \cdot S \cdot \log K$.

Czynnik $T \cdot S$ można zmniejszyć wykorzystując zoptymalizowane struktury danych do przeszukiwania przestrzeni (np. $kd$-drzewa), jednak one zazwyczaj radzą sobie dobrze tylko w niewielkiej liczbie wymiarów.

W praktyce, na naprawdę dużych zbiorach danych, stosuje się zazwyczaj algorytmy aproksymujące, które minimalnie zmniejszają zgodność klasyfikacji z oryginalną ideą algorytmu w zamian za znaczne zmniejszenie wymaganej liczby porównań.

**Wybór metryki**

- Euklidesowa

$$d(x, y) = \sqrt{\sum_{i = 1}^n (x_i - y_i)^2}$$

- Manhattan

$$d(x, y) = \sum_{i = 1}^n |x_i - y_i|$$

- Cosinusowa

$$\text{cossimilarity}(x, y) = \frac{x \cdot y}{ ||x|| \cdot ||y|| } $$

- Hamminga
$$H(x, y) = \sum_{i = 1}^n x_i \neq y_i.$$

**Ważony KNN**

Algorytm KNN można bardzo prosto uogólnić przypisując sąsiadom różne *wagi* na etapie wyboru najliczniejszej klasy. W najprostszej wersji algorytmu wszystkim sąsiadom przypisujemy stałą wagę $\frac{1}{K}$.

$d_i$ - dystans $i$-tego sąsiada od aktualnie rozpatrywanego obiektu 

W *ważonej* wersji KNN każdemu sąsiadowi przypisujemy wagę proporcjonalną do jego dystansu od rozpatrywanego obiektu
$$\frac{\frac{1}{d_i}}{\sum_{i = 1}^{K} \frac{1}{d_i}}.$$

In [ ]:
# imports
from typing import Literal
import requests
import gzip
import numpy as np
import io
import struct
import matplotlib.pyplot as plt

In [ ]:
def _download_and_decompress(url):
    print(f"Downloading {url}...")
    response = requests.get(url)
    with gzip.GzipFile(fileobj=io.BytesIO(response.content)) as f:
        return f.read()

def _read_images_from_bytes(data):
    _, num_images, rows, cols = struct.unpack(">IIII", data[:16])
    images = np.frombuffer(data[16:], dtype=np.uint8)
    images = images.reshape(num_images, rows, cols)
    return images

def _read_labels_from_bytes(data):
    labels = np.frombuffer(data[8:], dtype=np.uint8)
    return labels

def _parse_raw_data(url, func):
    raw_data = _download_and_decompress(url)
    return func(raw_data)

def download_set(kind: Literal["train", "test"]):
    if kind == "test":
        kind = "t10k" # Great naming.
    url_images = f"https://storage.googleapis.com/cvdf-datasets/mnist/{kind}-images-idx3-ubyte.gz"
    url_labels = f"https://storage.googleapis.com/cvdf-datasets/mnist/{kind}-labels-idx1-ubyte.gz"

    return _parse_raw_data(url_images, _read_images_from_bytes), _parse_raw_data(url_labels, _read_labels_from_bytes)

In [ ]:
train_data, train_labels = download_set("train")
train_data, train_labels = train_data[:1000], train_labels[:1000]
test_data, test_labels = download_set("test")
test_data, test_labels = test_data[:200], test_labels[:200]

# **W rozwiązaniach zadań 1-7 nie wolno importować nowych bibliotek!**

**Zadanie 1.**

Narysuj (na gridzie 2x5) po jednym przykładzie każdej z cyfr z danych treningowych.

In [ ]:
pass

**Zadanie 2.**

Zaimplementuj klasę `KNNClassifier`, która w konstruktorze przyjmuje treningowe obrazki, ich etykiety oraz parametr $K$.

Klasa ta powinna mieć funkcję `classify`, która przyjmuje obrazek ze zbioru testowego, znajduje $K$ najbliższych sąsiadów i zwraca najbardziej popularną etykietę spośród tych sąsiadów. W przypadku remisów może zwracać dowolną najpopularniejszą etykietę.

Możesz użyć dowolnej metryki spośród tych wymienionych w części teoretycznej.

Pamiętaj, że obrazki są dwuwymiarowymi tablicami.

In [ ]:
pass

**Zadanie 3.**

Stwórz instancję `KNNClassifier` z danymi treningowymi dla `K` równego 3. Dokonaj klasyfikacji zbioru testowego i oblicz accuracy.

In [ ]:
pass

**Zadanie 4.**

Powtórz eksperyment dla `K` z zakresu 1 do 20. Narysuj wykres jak się zmienia accuracy w zależności od $K$.

In [ ]:
pass

**Zadanie 5.**

Odczytaj z wykresu optymalne `K` względem accuracy. Wybierz 20 losowych obrazków, dla których Twój klasyfikator się pomylił. Narysuj je (jako grid 4x5) i podpisz poprawną etykietą oraz tą wskazaną przez klasyfikator.

In [ ]:
pass

**Zadanie 6.**

Zaimplementuj klasę `WeightedKNNClassifier`, która działa analogicznie do klasy `KNNClassifier`, ale implementuje *ważoną* wersję algorytmu KNN.

In [ ]:
pass

**Zadanie 7.**

Nanieś accuracy zwykłego oraz ważonego klasyfikatora KNN dla $K$ z przediału od 1 do 20 i oceń, który z tych klasyfikatorów sprawdził się lepiej dla tych danych.

In [ ]:
pass

**Zadanie 8.**

Wykorzystaj bibliotekę `scikit-learn` do znalezienia najlepszych hiperparametrów dla algorytmu KNN na zbiorze danych 'iris' wykorzystując podejście grid search.

Sprawdź następujące zbiory hiperparametrów:
- $K \in [1, 20]$,
- algorytm KNN zwykły albo ważony,
- metryka euklidesowa lub Manhattan.

Zbiór danych podziel na zbiór treningowy i testowy w proporcji 1:1.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

pass